In [ ]:
import holoviews as hv
import numpy as np
import panel as pn
from PIL import Image

hv.extension('bokeh')

In [ ]:
x = np.array(Image.open("/home/joe/Documents/electricmayhem_tutorials/data/flower2.png"))
imshape = x.shape
print(imshape)
img = hv.RGB(x, bounds=[0, 0, 320, -240])#.opts(invert_yaxis=True)
img

In [ ]:
#curve = hv.Curve(np.random.randn(50).cumsum())
imshape = x.shape
a = imshape[1]/2 - imshape[1]/4
b = imshape[1]/2 + imshape[1]/4
c = -1*imshape[0]/2 - imshape[0]/4
d = -1*imshape[0]/2 + imshape[0]/4

curve = hv.Curve(([a,a,b,b], [d,c,c,d]))
curve

In [ ]:
curve_annotator = hv.annotate.instance()

In [ ]:
img*curve_annotator(curve, annotations={"Label":str})

In [ ]:
curve_annotator.annotated.dframe()

In [ ]:
import matplotlib.pyplot as plt
import torch
import kornia.geometry, kornia.augmentation

In [ ]:
im = np.array(Image.open("/home/joe/Documents/electricmayhem_tutorials/data/flower2.png")).astype(np.float32)/255
imt = torch.tensor(im).permute(2,0,1)[:3,:,:]
print(imt.shape)
plt.imshow(im);

In [ ]:
aug = kornia.augmentation.RandomAffine(scale=(0.5,1.5), translate=0.25, degrees=10)
im1 = aug(imt).detach()
im2 = aug(imt).detach()
plt.subplot(121)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.subplot(122)
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy());

In [ ]:
aug._params

In [ ]:
#reg = kornia.geometry.ImageRegistrator(num_iterations=1000)
reg = kornia.geometry.ImageRegistrator("homography", 
                                       num_iterations=1000,
                                      pyramid_levels=7, # 5
                                       tolerance=1e-8
                                      )

In [ ]:
%%time
model = reg.register(im1, im2)

In [ ]:
#kornia.geometry.ImageRegistrator?

In [ ]:
model

In [ ]:
#invmodel = model = reg.register(im2, im1)

In [ ]:
model.shape

In [ ]:
warped = kornia.geometry.homography_warp(im1, model, (240,320), normalized_homography=True)

In [ ]:
plt.subplot(121)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.title("original")
plt.subplot(122)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy());

In [ ]:
plt.subplot(121)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.subplot(122)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy());

In [ ]:
img = hv.RGB(im1.squeeze(0).permute(1,2,0).numpy(), bounds=[0, 0, 320, -240])#.opts(invert_yaxis=True)
imshape = x.shape
a = imshape[1]/2 - imshape[1]/4
b = imshape[1]/2 + imshape[1]/4
c = -1*imshape[0]/2 - imshape[0]/4
d = -1*imshape[0]/2 + imshape[0]/4
curve = hv.Curve(([a,a,b,b], [d,c,c,d]))
curve_annotator = hv.annotate.instance()
img*curve_annotator(curve, annotations={"Label":str})

In [ ]:
curve_annotator.annotated.dframe()

In [ ]:
# directly put pixels through
df = curve_annotator.annotated.dframe()
lab = torch.tensor([-1*df.y.values, df.x.values]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model, lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(lab[0,:,1].numpy(), lab[0,:,0].numpy())
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(lab[0,:,1].numpy(), lab[0,:,0].numpy())
plt.plot(transformed_lab[0,:,1].numpy(), transformed_lab[0,:,0].numpy())
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(transformed_lab[0,:,1].numpy(), transformed_lab[0,:,0].numpy());

In [ ]:
# attempt conversion to normalized coordinates to (-1,1) first
df = curve_annotator.annotated.dframe()
lab = torch.tensor([-1*df.y.values/120-1, df.x.values/160-1]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model, lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*lab[0,:,1].numpy()+160, 120*lab[0,:,0].numpy()+120)
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(160*lab[0,:,1].numpy()+160, 120*lab[0,:,0].numpy()+120)
plt.plot(160*transformed_lab[0,:,1].numpy()+160, 120*transformed_lab[0,:,0].numpy()+120)
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*transformed_lab[0,:,1].numpy()+160, 120*transformed_lab[0,:,0].numpy()+120);

In [ ]:
# attempt conversion to normalized coordinates to (-1,1) first and use inverse homography
# (suggested in issue 849)
df = curve_annotator.annotated.dframe()
lab = torch.tensor([-1*df.y.values/120-1, df.x.values/160-1]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model.inverse(), lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*lab[0,:,1].numpy()+160, 120*lab[0,:,0].numpy()+120)
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(160*lab[0,:,1].numpy()+160, 120*lab[0,:,0].numpy()+120)
plt.plot(160*transformed_lab[0,:,1].numpy()+160, 120*transformed_lab[0,:,0].numpy()+120)
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*transformed_lab[0,:,1].numpy()+160, 120*transformed_lab[0,:,0].numpy()+120);

In [ ]:
# attempt conversion to normalized coordinates to (0,1) first
df = curve_annotator.annotated.dframe()
lab = torch.tensor([-1*df.y.values/240, df.x.values/320]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model, lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(320*lab[0,:,1].numpy(), 240*lab[0,:,0].numpy())
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(320*lab[0,:,1].numpy(), 240*lab[0,:,0].numpy())
plt.plot(320*transformed_lab[0,:,1].numpy(), 240*transformed_lab[0,:,0].numpy())
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(320*transformed_lab[0,:,1].numpy(), 240*transformed_lab[0,:,0].numpy());

In [ ]:
# directly put pixels through inverse transform
df = curve_annotator.annotated.dframe()
lab = torch.tensor([-1*df.y.values, df.x.values]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model.inverse(), lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(lab[0,:,1].numpy(), lab[0,:,0].numpy())
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(lab[0,:,1].numpy(), lab[0,:,0].numpy())
plt.plot(transformed_lab[0,:,1].numpy(), transformed_lab[0,:,0].numpy())
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(transformed_lab[0,:,1].numpy(), transformed_lab[0,:,0].numpy());

In [ ]:
# directly put pixels through, swapping x and y channels
df = curve_annotator.annotated.dframe()
lab = torch.tensor([df.x.values, -1*df.y.values]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model, lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(lab[0,:,0].numpy(), lab[0,:,1].numpy())
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(lab[0,:,0].numpy(), lab[0,:,1].numpy())
plt.plot(transformed_lab[0,:,0].numpy(), transformed_lab[0,:,1].numpy())
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(transformed_lab[0,:,0].numpy(), transformed_lab[0,:,1].numpy());

In [ ]:
# attempt conversion to normalized coordinates to (-1,1) first and use inverse homography
# (suggested in issue 849)
# but this time also swap x and y channels
df = curve_annotator.annotated.dframe()
lab = torch.tensor([df.x.values/160-1, -1*df.y.values/120-1]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model.inverse(), lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*lab[0,:,0].numpy()+160, 120*lab[0,:,1].numpy()+120)
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(160*lab[0,:,0].numpy()+160, 120*lab[0,:,1].numpy()+120)
plt.plot(160*transformed_lab[0,:,0].numpy()+160, 120*transformed_lab[0,:,1].numpy()+120)
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*transformed_lab[0,:,0].numpy()+160, 120*transformed_lab[0,:,1].numpy()+120);

In [ ]:
kornia.geometry.homography.transform_points??

In [ ]:
kornia.geometry.linalg.convert_points_to_homogeneous?